<a href="https://vigneashpandiyan.github.io/publications/Codes/" target="_blank" rel="noopener noreferrer">
  <img src="https://vigneashpandiyan.github.io/images/Link.png"
       style="max-width: 800px; width: 100%; height: auto;">
</a>

In [ ]:
import numpy as np
import cv2 # OpenCV-Python
%matplotlib inline
import matplotlib.pyplot as plt
import time

In [ ]:
# Load an image
beaver = cv2.imread('data/beaver.png')
plt.imshow(cv2.cvtColor(beaver, cv2.COLOR_BGR2RGB))

In [ ]:
# Convert image color(BGR->Grayscale)
gray = cv2.cvtColor(beaver, cv2.COLOR_BGR2GRAY)
# You can convert the image when calling cv2.imread()
# gray = cv2.imread('images/beaver.png', cv2.IMREAD_GRAYSCALE)

print(str(beaver.shape) + " => " + str(gray.shape))
plt.imshow(gray, cmap='gray')

In [ ]:
# SIFT feature detector/descriptor
sift = cv2.SIFT_create()
kp, des = sift.detectAndCompute(beaver, None)

In [ ]:
start_time = time.time()
print('Elapsed time: %.6fs' % (time.time() - start_time))

In [ ]:
# Display the SIFT features
beaver_sift = cv2.drawKeypoints(beaver, kp, None)
plt.imshow(cv2.cvtColor(beaver_sift, cv2.COLOR_BGR2RGB))

In [ ]:
# Display the rich SIFT features
beaver_sift2 = cv2.drawKeypoints(beaver, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(cv2.cvtColor(beaver_sift2, cv2.COLOR_BGR2RGB))

In [ ]:
# Inspect the keypoints
print(type(kp))
print(len(kp))

In [ ]:
print(type(kp[0]))
print(dir(kp[0]))

In [ ]:
# A keypoint's property
# kp is sorted by scale of the keypoints
print(kp[-1].angle)      # orientation
print(kp[-1].class_id)
print(kp[-1].octave)
print(kp[-1].pt)         # (x, y)
print(kp[-1].response)
print(kp[-1].size)       # scale

In [ ]:
# Extract SIFT feature from the (gray) image and detected keypoints
start_time = time.time()
kp, des = sift.compute(gray, kp)
print('Elapsed time: %.6fs' % (time.time() - start_time))

# SIFT keypoints and descriptors at the same time
# start_time = time.time()
# kp, des = sift.detectAndCompute(gray, None)
# print('Elapsed time: %.6fs' % (time.time() - start_time))

In [ ]:
# Inspect the descriptors
print(type(des))
print(des.shape)
print(des.dtype)


In [ ]:
print(len(des[0, :]))
print(des[0, :])

In [ ]:
# Open and show images
img1 = cv2.imread('data/box.png')
img2 = cv2.imread('data/box_in_scene.png')

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))

In [ ]:

if img1 is None or img2 is None:
    raise ValueError("img1/img2 is None. Check cv2.imread paths.")

# --- Grayscale ---
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# --- Create SIFT (works on most OpenCV 4.x builds) ---
sift = cv2.SIFT_create()

t0 = time.time()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)
print(f"Elapsed time: {(time.time()-t0):.6f}s")

print("Image 1 - features:", 0 if des1 is None else des1.shape[0])
print("Image 2 - features:", 0 if des2 is None else des2.shape[0])

if des1 is None or des2 is None:
    raise RuntimeError("No descriptors found in one of the images. Try a different image or adjust detector params.")

# --- BFMatcher for SIFT uses NORM_L2 ---
bf = cv2.BFMatcher(cv2.NORM_L2)
matches = bf.knnMatch(des1, des2, k=2)
print("knn matches:", len(matches))


In [ ]:
# Inspect matcher results

print(type(matches))
print(len(matches))
print(type(matches[0]))
print(len(matches[0]))




In [ ]:
print(matches[0][0].distance)
print(matches[0][0].queryIdx)
print(matches[0][0].trainIdx)
print(matches[0][0].imgIdx)

print(matches[0][1].distance)
print(matches[0][1].queryIdx)
print(matches[0][1].trainIdx)
print(matches[0][1].imgIdx)


In [ ]:
# Apply ratio test as in David Rowe's paper
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)
print('%d matches' % len(good_matches))

In [ ]:
# Display matches
img3 = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None)
plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))